## ETL on Sparkify Event Data

This Jupyter notebook performs ETL on the Sparkify event data in JSON format. The purpose of this notebook is to clean and transform the data so that it can be used for further analysis.

### Step 1: Load the Data

The first step is to load the data into PySpark using the `spark.read.json()` function. We will load the data from the `sparkify_event_data.json` file and store it in a DataFrame called `df`.

### Step 2: Data Cleaning

The next step is to clean the data. We will perform the following cleaning steps:

#### Remove Null Values

We will remove any rows that contain null values.

#### Select Users that had the 'paid' Level

We will select only the users who had a 'paid' level using the PySpark SQL functions. We will create a new DataFrame called `df_filter` that contains only the relevant rows.

### Step 3: Create a Table of Occurrences

Next, we will create a table that counts the number of occurrences for the cleaned group using the PySpark SQL functions. We will create a new DataFrame called `data` that contains the counts.

### Step 4: Convert Gender and Churn into Numbers
Then, we will convert the genders into a numeric form, where `male` will ve assgined a value of `1` and `female` a value of `0`. In the case of churning, a column named `label` will be created if a `submit_downgrade` is greater than `1`.

### Step 5: Store the Data

Finally, we will store the data as a single CSV file in the `features/` folder. 

## Conclusion

This notebook demonstrates how to perform ETL on a big dataset using PySpark and SQL. By following the steps outlined in this notebook, you can clean and transform your data for further analysis.


In [1]:
# Starter code
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, count, udf
from pyspark.sql.functions import max as fmax, min as fmin
from pyspark.sql.types import IntegerType

In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

23/03/11 16:04:31 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.248.232 instead (on interface wlp110s0)
23/03/11 16:04:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/11 16:04:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/11 16:04:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [4]:
# Read in full sparkify dataset
# Full dataset
# event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
event_data = "sparkify_event_data.json"
# Mini dataset
#event_data = "mini_sparkify_event_data.json"
df = spark.read.json(event_data)

In [5]:
# We create a temp table named sparkify_data
df.createOrReplaceTempView("sparkify_data")

In [6]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
number_of_users = spark.sql("""
        SELECT COUNT(DISTINCT(userId)) FROM sparkify_data
        """).collect()

In [8]:
print(f'Number of users = {number_of_users[0][0]}')

Number of users = 22278


In [9]:
spark.sql("""
        WITH events AS(
        SELECT userID, page, COUNT(page) FROM sparkify_data
        GROUP BY userId, page)
        
        SELECT page, COUNT(page) AS count FROM events
        GROUP BY page
        """)

page,count
Cancel,5003
Submit Downgrade,5103
Thumbs Down,20031
Home,22073
Downgrade,15209
Roll Advert,20068
Logout,21160
Save Settings,12237
Cancellation Conf...,5003
About,14461


In [10]:
spark.sql("""
            SELECT COUNT(CASE WHEN artist IS NULL THEN 1 END) AS artist_null_count,
            COUNT(CASE WHEN auth IS NULL THEN 1 END) AS auth_null_count,
            COUNT(CASE WHEN firstName IS NULL THEN 1 END) AS firstName_null_count,
            COUNT(CASE WHEN gender IS NULL THEN 1 END) AS gender_null_count,
            COUNT(CASE WHEN itemInSession IS NULL THEN 1 END) AS itemInSession_null_count,
            COUNT(CASE WHEN lastName IS NULL THEN 1 END) AS lastName_null_count,
            COUNT(CASE WHEN length IS NULL THEN 1 END) AS length_null_count,
            COUNT(CASE WHEN level IS NULL THEN 1 END) AS level_null_count,
            COUNT(CASE WHEN location IS NULL THEN 1 END) AS location_null_count,
            COUNT(CASE WHEN method IS NULL THEN 1 END) AS method_null_count,
            COUNT(CASE WHEN page IS NULL THEN 1 END) AS page_null_count,
            COUNT(CASE WHEN registration IS NULL THEN 1 END) AS registration_null_count,
            COUNT(CASE WHEN sessionId IS NULL THEN 1 END) AS sessionId_null_count,
            COUNT(CASE WHEN song IS NULL THEN 1 END) AS song_null_count,
            COUNT(CASE WHEN status IS NULL THEN 1 END) AS status_null_count,
            COUNT(CASE WHEN ts IS NULL THEN 1 END) AS ts_null_count,
            COUNT(CASE WHEN userAgent IS NULL THEN 1 END) AS userAgent_null_count,
            COUNT(CASE WHEN userId IS NULL THEN 1 END) AS userId_null_count
            FROM sparkify_data
    """)

artist_null_count,auth_null_count,firstName_null_count,gender_null_count,itemInSession_null_count,lastName_null_count,length_null_count,level_null_count,location_null_count,method_null_count,page_null_count,registration_null_count,sessionId_null_count,song_null_count,status_null_count,ts_null_count,userAgent_null_count,userId_null_count
5408927,0,778479,778479,0,778479,5408927,0,778479,0,0,778479,0,5408927,0,0,778479,0


In [11]:
# We want to remove null values and free users from our dataset
entries_to_remove = spark.sql("""
        SELECT COUNT(*) FROM sparkify_data
        WHERE gender IS NULL OR level != 'paid'
        """).collect()

removed_percentage = entries_to_remove[0][0]/df.count()*100

print(f'The number of entries to remove is = {entries_to_remove[0][0]}')
print(f'The percentage of data removed is = {removed_percentage:.4n}%')


The number of entries to remove is = 6186719
The percentage of data removed is = 23.56%


In [12]:
df_filter = spark.sql("""
            SELECT * FROM sparkify_data
            WHERE gender IS NOT NULL AND level = 'paid'
            """)

In [13]:
df_filter

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
Popol Vuh,Logged In,Shlok,M,278,Johnson,524.32934,paid,Dallas-Fort Worth...,PUT,NextSong,1533734541000,22683,Ich mache einen S...,200,1538352001000,"""Mozilla/5.0 (Win...",1749042
Los Bunkers,Logged In,Vianney,F,9,Miller,238.39302,paid,San Francisco-Oak...,PUT,NextSong,1537500318000,20836,MiÃÂ©ntele,200,1538352002000,"""Mozilla/5.0 (Mac...",1563081
Lush,Logged In,Vina,F,109,Bailey,140.35546,paid,"Hilo, HI",PUT,NextSong,1536414505000,4593,Baby Talk,200,1538352002000,Mozilla/5.0 (Maci...,1697168
Barry Tuckwell/Ac...,Logged In,Andres,M,71,Foley,277.15873,paid,"Watertown, SD",PUT,NextSong,1534386660000,6370,Horn Concerto No....,200,1538352003000,"""Mozilla/5.0 (Mac...",1222580
Yes,Logged In,Aaliyah,F,21,Ramirez,1121.25342,paid,Baltimore-Columbi...,PUT,NextSong,1537381415000,22316,Close To The Edge...,200,1538352003000,"""Mozilla/5.0 (Win...",1714398
MGMT,Logged In,Ryan,M,8,Williams,229.25016,paid,Cape Coral-Fort M...,PUT,NextSong,1537602560000,23924,Electric Feel,200,1538352004000,"""Mozilla/5.0 (Win...",1010522
The Cranberries,Logged In,Michael,M,13,Riley,183.87546,paid,Albany-Schenectad...,PUT,NextSong,1533220062000,3992,The Concept,200,1538352006000,"""Mozilla/5.0 (Win...",1605667
Johnny Cash with ...,Logged In,Riley,F,300,Hernandez,199.81016,paid,"Reading, PA",PUT,NextSong,1537618545000,14853,Nine Pound Hammer,200,1538352006000,"""Mozilla/5.0 (Mac...",1032628
Los Prisioneros,Logged In,Brayden,M,95,Armstrong,248.78975,paid,Dallas-Fort Worth...,PUT,NextSong,1534635513000,23917,La Voz De Los '80,200,1538352007000,"""Mozilla/5.0 (Win...",1611729
The White Stripes,Logged In,Brooke,F,6,Kirby,126.40608,paid,Los Angeles-Long ...,PUT,NextSong,1531817572000,14085,Good To Me,200,1538352008000,"""Mozilla/5.0 (Win...",1497881


In [14]:
# We create a temp table named data
df_filter.createOrReplaceTempView("data")

In [15]:
spark.sql("""
        WITH events AS(
        SELECT userID, page, COUNT(page) FROM data
        GROUP BY userId, page)
        
        SELECT page, COUNT(page) AS count FROM events
        GROUP BY page
        """)

page,count
Cancel,3424
Submit Downgrade,5103
Thumbs Down,14850
Home,16133
Downgrade,15209
Roll Advert,8956
Logout,15082
Save Settings,9273
Cancellation Conf...,3424
About,11129


In [16]:
data = spark.sql("""
        SELECT BIGINT(userId),
        gender,
        COUNT(page) AS n_pages,
        COUNT(IF(page='Submit Downgrade',1,NULL)) AS submit_downgrade,
        COUNT(IF(page='Thumbs Down',1,NULL)) AS thumbs_down,
        COUNT(IF(page='Home',1,NULL)) AS home,
        COUNT(IF(page='Downgrade',1,NULL)) AS downgrade,
        COUNT(IF(page='Roll Advert',1,NULL)) AS roll_advert,
        COUNT(IF(page='Cancellation Confirmation',1,NULL)) AS cancellation,
        COUNT(IF(page='About',1,NULL)) AS about,
        COUNT(IF(page='Submit Registration',1,NULL)) AS submit_registration,
        COUNT(IF(page='Cancel',1,NULL)) AS cancel,
        COUNT(IF(page='Login',1,NULL)) AS login,
        COUNT(IF(page='Register',1,NULL)) AS register,
        COUNT(IF(page='Add to Playlist',1,NULL)) AS add_playlist,
        COUNT(IF(page='NextSong',1,NULL)) AS nextsong,
        COUNT(IF(page='Thumbs Up',1,NULL)) AS thumbs_up,
        COUNT(IF(page='Error',1,NULL)) AS error,
        COUNT(IF(page='Submit Upgrade',1,NULL)) AS submit_upgrade,
        CEILING(SUM(length)) AS total_length
        FROM sparkify_data 
        GROUP BY userId,gender

    """)

In [17]:
churn = udf(lambda x: 1 if x>0 else 0, IntegerType())
gender = udf(lambda x: 1 if x=='M' else 0, IntegerType())

In [18]:
data = data.withColumn('label', churn('submit_downgrade'))
data = data.withColumn('gender', gender('gender'))

In [19]:
data.coalesce(1).write.options(header='True', delimiter=',').csv("features")